# E-commerce Business Analytics Dashboard

## Executive Summary
This notebook provides a comprehensive analysis of e-commerce business performance, focusing on revenue trends, customer behavior, operational efficiency, and market insights. The analysis is designed to support data-driven business decisions through configurable metrics and visualizations.

## Business Objectives
- **Revenue Analysis**: Track revenue growth, seasonal trends, and performance drivers
- **Product Performance**: Identify top-performing categories and optimize product mix
- **Geographic Insights**: Understand regional market opportunities and expansion potential
- **Customer Experience**: Measure satisfaction and operational efficiency
- **Operational Excellence**: Monitor fulfillment rates and delivery performance

## Table of Contents
1. [Data Loading & Configuration](#data-loading--configuration)
2. [Data Preparation & Transformation](#data-preparation--transformation)
3. [Revenue Analysis](#revenue-analysis)
4. [Product Category Performance](#product-category-performance)
5. [Geographic Market Analysis](#geographic-market-analysis)
6. [Customer Experience Analysis](#customer-experience-analysis)
7. [Operational Performance](#operational-performance)
8. [Summary & Key Insights](#summary--key-insights)

## Configuration

### Analysis Period Configuration
Set the time periods for analysis. The default configuration compares 2023 performance against 2022 baseline.

In [ ]:
# Analysis Configuration
ANALYSIS_CONFIG = {
    'current_year': 2023,
    'comparison_year': 2022,
    'current_start_month': 1,
    'current_end_month': 12,
    'comparison_start_month': 1,
    'comparison_end_month': 12,
    'data_path': 'ecommerce_data'
}

# Display configuration
print("Analysis Configuration:")
print(f"Current Period: {ANALYSIS_CONFIG['current_year']} (Months {ANALYSIS_CONFIG['current_start_month']}-{ANALYSIS_CONFIG['current_end_month']})")
print(f"Comparison Period: {ANALYSIS_CONFIG['comparison_year']} (Months {ANALYSIS_CONFIG['comparison_start_month']}-{ANALYSIS_CONFIG['comparison_end_month']})")
print(f"Data Source: {ANALYSIS_CONFIG['data_path']}")

## Data Loading & Configuration

### Import Libraries and Modules

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Custom modules
from data_loader import (
    load_datasets, 
    clean_and_prepare_data, 
    create_sales_dataset, 
    filter_data_by_date_range,
    add_product_categories,
    add_customer_geography,
    add_review_data,
    get_data_summary
)

from business_metrics import (
    calculate_revenue_metrics,
    calculate_growth_metrics,
    calculate_monthly_trends,
    calculate_product_category_metrics,
    calculate_geographic_metrics,
    calculate_customer_experience_metrics,
    calculate_operational_metrics,
    generate_metrics_summary
)

# Visualization configuration
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Business color scheme
BUSINESS_COLORS = {
    'primary': '#1f77b4',      # Professional blue
    'success': '#2ca02c',      # Growth green
    'warning': '#ff7f0e',      # Alert orange
    'danger': '#d62728',       # Decline red
    'info': '#17a2b8',         # Information teal
    'neutral': '#6c757d'       # Neutral gray
}

print("Libraries and modules loaded successfully")

### Load Raw Data

In [ ]:
# Load all datasets
print("Loading e-commerce datasets...")
datasets = load_datasets(ANALYSIS_CONFIG['data_path'])

# Display dataset overview
print("\nDataset Overview:")
for name, df in datasets.items():
    print(f"{name.title()}: {df.shape[0]:,} rows × {df.shape[1]} columns")

### Data Dictionary

**Key Business Terms and Data Definitions:**

**Orders Data:**
- `order_id`: Unique identifier for each order
- `customer_id`: Unique identifier for each customer
- `order_status`: Current status (delivered, shipped, canceled, etc.)
- `order_purchase_timestamp`: When the order was placed
- `order_delivered_customer_date`: When delivered to customer

**Order Items Data:**
- `price`: Item price in USD
- `freight_value`: Shipping cost in USD
- `product_id`: Unique product identifier

**Products Data:**
- `product_category_name`: Product category classification
- `product_weight_g`: Product weight in grams

**Customer Data:**
- `customer_state`: State/region location
- `customer_city`: City location

**Reviews Data:**
- `review_score`: Customer rating (1-5 stars)
- `review_creation_date`: When review was submitted

**Calculated Metrics:**
- `Average Order Value (AOV)`: Total revenue ÷ number of orders
- `Delivery Speed`: Days from purchase to delivery
- `Satisfaction Rate`: Percentage of 4-5 star reviews

## Data Preparation & Transformation

### Clean and Process Raw Data

In [ ]:
# Clean and prepare all datasets
print("Cleaning and preparing datasets...")
clean_datasets = clean_and_prepare_data(datasets)

# Extract individual datasets for easier reference
orders_df = clean_datasets['orders']
order_items_df = clean_datasets['order_items']
products_df = clean_datasets['products']
customers_df = clean_datasets['customers']
reviews_df = clean_datasets['reviews']

print("Data cleaning completed successfully")

# Display data quality summary
print("\nData Quality Summary:")
for name, df in clean_datasets.items():
    missing_pct = (df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100
    print(f"{name.title()}: {missing_pct:.1f}% missing values")

### Create Consolidated Sales Dataset

In [ ]:
# Create base sales dataset (delivered orders only)
print("Creating consolidated sales dataset...")
sales_df = create_sales_dataset(clean_datasets, order_status_filter='delivered')

# Add product categories
sales_df = add_product_categories(sales_df, products_df)

# Add customer geography
sales_df = add_customer_geography(sales_df, orders_df, customers_df)

# Add review data
sales_df = add_review_data(sales_df, reviews_df)

print(f"Consolidated dataset created: {sales_df.shape[0]:,} records")
print(f"Date range: {sales_df['order_purchase_timestamp'].min().strftime('%Y-%m-%d')} to {sales_df['order_purchase_timestamp'].max().strftime('%Y-%m-%d')}")

### Filter Data by Analysis Periods

In [ ]:
# Filter data for current analysis period
current_period_df = filter_data_by_date_range(
    sales_df,
    start_year=ANALYSIS_CONFIG['current_year'],
    end_year=ANALYSIS_CONFIG['current_year'],
    start_month=ANALYSIS_CONFIG['current_start_month'],
    end_month=ANALYSIS_CONFIG['current_end_month']
)

# Filter data for comparison period
comparison_period_df = filter_data_by_date_range(
    sales_df,
    start_year=ANALYSIS_CONFIG['comparison_year'],
    end_year=ANALYSIS_CONFIG['comparison_year'],
    start_month=ANALYSIS_CONFIG['comparison_start_month'],
    end_month=ANALYSIS_CONFIG['comparison_end_month']
)

print(f"Current period dataset: {current_period_df.shape[0]:,} records")
print(f"Comparison period dataset: {comparison_period_df.shape[0]:,} records")

# Display sample of current period data
print("\nCurrent Period Sample:")
display(current_period_df[['order_id', 'product_category_name', 'price', 'customer_state', 'review_score']].head())

## Revenue Analysis

### Overall Revenue Performance

In [ ]:
# Calculate revenue metrics for both periods
current_metrics = calculate_revenue_metrics(current_period_df)
comparison_metrics = calculate_revenue_metrics(comparison_period_df)
growth_metrics = calculate_growth_metrics(current_period_df, comparison_period_df)

# Display key metrics
print("REVENUE PERFORMANCE SUMMARY")
print("=" * 50)
print(f"Current Period ({ANALYSIS_CONFIG['current_year']}):")
print(f"  Total Revenue: ${current_metrics['total_revenue']:,.2f}")
print(f"  Total Orders: {current_metrics['total_orders']:,}")
print(f"  Average Order Value: ${current_metrics['avg_order_value']:,.2f}")
print(f"  Average Items per Order: {current_metrics['avg_items_per_order']:.1f}")

print(f"\nComparison Period ({ANALYSIS_CONFIG['comparison_year']}):")
print(f"  Total Revenue: ${comparison_metrics['total_revenue']:,.2f}")
print(f"  Total Orders: {comparison_metrics['total_orders']:,}")
print(f"  Average Order Value: ${comparison_metrics['avg_order_value']:,.2f}")

print(f"\nYEAR-OVER-YEAR GROWTH:")
print(f"  Revenue Growth: {growth_metrics['revenue_growth_pct']:+.2f}%")
print(f"  Order Growth: {growth_metrics['order_growth_pct']:+.2f}%")
print(f"  AOV Growth: {growth_metrics['aov_growth_pct']:+.2f}%")

### Monthly Revenue Trends

In [ ]:
# Calculate monthly trends for current year
monthly_trends = calculate_monthly_trends(current_period_df, year_filter=ANALYSIS_CONFIG['current_year'])

# Create monthly revenue trend visualization
fig = go.Figure()

# Add revenue line
fig.add_trace(go.Scatter(
    x=monthly_trends['month'],
    y=monthly_trends['total_revenue'],
    mode='lines+markers',
    name='Monthly Revenue',
    line=dict(color=BUSINESS_COLORS['primary'], width=3),
    marker=dict(size=8)
))

# Update layout
fig.update_layout(
    title=f'Monthly Revenue Trend - {ANALYSIS_CONFIG["current_year"]}',
    xaxis_title='Month',
    yaxis_title='Revenue (USD)',
    yaxis_tickformat='$,.0f',
    hovermode='x unified',
    template='plotly_white',
    height=400
)

fig.show()

# Display monthly growth rates
print("MONTHLY GROWTH RATES:")
for _, row in monthly_trends.iterrows():
    growth = row['revenue_mom_growth']
    if pd.notna(growth):
        print(f"  Month {int(row['month'])}: {growth:+.2f}%")

### Revenue Distribution Analysis

In [ ]:
# Calculate order value distribution
order_values = current_period_df.groupby('order_id')['price'].sum()

# Create revenue distribution visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Histogram of order values
ax1.hist(order_values, bins=50, color=BUSINESS_COLORS['primary'], alpha=0.7, edgecolor='white')
ax1.set_title('Order Value Distribution', fontsize=14, fontweight='bold')
ax1.set_xlabel('Order Value (USD)')
ax1.set_ylabel('Number of Orders')
ax1.grid(True, alpha=0.3)

# Box plot of order values
ax2.boxplot(order_values, vert=True, patch_artist=True, 
            boxprops=dict(facecolor=BUSINESS_COLORS['primary'], alpha=0.7))
ax2.set_title('Order Value Distribution Summary', fontsize=14, fontweight='bold')
ax2.set_ylabel('Order Value (USD)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Display percentile information
percentiles = [25, 50, 75, 90, 95]
print("ORDER VALUE PERCENTILES:")
for p in percentiles:
    value = np.percentile(order_values, p)
    print(f"  {p}th percentile: ${value:.2f}")

## Product Category Performance

### Data Structure Check

In [ ]:
# Debug: Check data structure before category analysis
print("DEBUGGING DATA STRUCTURE:")
print(f"Products DataFrame columns: {list(products_df.columns)}")
print(f"Current period DataFrame columns: {list(current_period_df.columns)}")
print(f"Products DataFrame shape: {products_df.shape}")
print(f"Current period DataFrame shape: {current_period_df.shape}")

# Check if product_category_name exists in either dataset
if 'product_category_name' in products_df.columns:
    print(f"Product categories found: {products_df['product_category_name'].nunique()} unique categories")
    print(f"Sample categories: {products_df['product_category_name'].unique()[:5]}")
else:
    print("WARNING: product_category_name not found in products_df")

if 'product_category_name' in current_period_df.columns:
    print(f"Categories in current period data: {current_period_df['product_category_name'].nunique()} unique")
else:
    print("WARNING: product_category_name not found in current_period_df")

# Check product_id presence and overlap
if 'product_id' in products_df.columns and 'product_id' in current_period_df.columns:
    products_in_sales = current_period_df['product_id'].nunique()
    products_in_catalog = products_df['product_id'].nunique()
    overlap = len(set(current_period_df['product_id']) & set(products_df['product_id']))
    print(f"Product IDs - Sales: {products_in_sales}, Catalog: {products_in_catalog}, Overlap: {overlap}")

In [ ]:
# Calculate category metrics for current period
category_metrics = calculate_product_category_metrics(current_period_df, products_df)

# Check if we got valid results
if category_metrics.empty:
    print("ERROR: No category metrics calculated. Check data structure above.")
    # Create a fallback analysis without categories
    print("\nFALLBACK: Analyzing without product categories")
    print(f"Total revenue (all products): ${current_period_df['price'].sum():,.2f}")
    print(f"Total products sold: {current_period_df['product_id'].nunique():,}")
else:
    # Display top categories
    print("TOP PRODUCT CATEGORIES BY REVENUE:")
    print("=" * 50)
    for idx, row in category_metrics.head(10).iterrows():
        print(f"{row['product_category_name'].title():25} ${row['total_revenue']:>10,.2f} ({row['revenue_share_pct']:4.1f}%)")
    
    # Create category revenue visualization
    top_categories = category_metrics.head(10)
    
    fig = go.Figure(data=[
        go.Bar(
            y=top_categories['product_category_name'].str.title(),
            x=top_categories['total_revenue'],
            orientation='h',
            marker_color=BUSINESS_COLORS['primary'],
            text=[f'${x:,.0f}' for x in top_categories['total_revenue']],
            textposition='outside'
        )
    ])
    
    fig.update_layout(
        title=f'Top 10 Product Categories by Revenue - {ANALYSIS_CONFIG["current_year"]}',
        xaxis_title='Revenue (USD)',
        yaxis_title='Product Category',
        yaxis={'categoryorder': 'total ascending'},
        template='plotly_white',
        height=500,
        margin=dict(l=200)
    )
    
    fig.show()

### Category Performance Metrics

In [ ]:
# Create category performance dashboard
if not category_metrics.empty:
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Revenue Share', 'Average Order Value', 'Items per Order', 'Revenue per Product'),
        specs=[[{"type": "pie"}, {"type": "bar"}],
               [{"type": "bar"}, {"type": "bar"}]]
    )

    # Revenue share pie chart
    top5_categories = category_metrics.head(5)
    others_revenue = category_metrics.iloc[5:]['total_revenue'].sum() if len(category_metrics) > 5 else 0
    pie_data = pd.concat([
        top5_categories[['product_category_name', 'total_revenue']],
        pd.DataFrame([{'product_category_name': 'Others', 'total_revenue': others_revenue}])
    ]) if others_revenue > 0 else top5_categories[['product_category_name', 'total_revenue']]

    fig.add_trace(go.Pie(
        labels=pie_data['product_category_name'].str.title(),
        values=pie_data['total_revenue'],
        textinfo='label+percent'
    ), row=1, col=1)

    # Average Order Value by category
    fig.add_trace(go.Bar(
        x=top5_categories['avg_order_value'],
        y=top5_categories['product_category_name'].str.title(),
        orientation='h',
        marker_color=BUSINESS_COLORS['success'],
        name='AOV'
    ), row=1, col=2)

    # Items per order
    fig.add_trace(go.Bar(
        x=top5_categories['items_per_order'],
        y=top5_categories['product_category_name'].str.title(),
        orientation='h',
        marker_color=BUSINESS_COLORS['info'],
        name='Items/Order'
    ), row=2, col=1)

    # Revenue per product
    fig.add_trace(go.Bar(
        x=top5_categories['revenue_per_product'],
        y=top5_categories['product_category_name'].str.title(),
        orientation='h',
        marker_color=BUSINESS_COLORS['warning'],
        name='Revenue/Product'
    ), row=2, col=2)

    fig.update_layout(
        title_text=f"Product Category Performance Dashboard - {ANALYSIS_CONFIG['current_year']}",
        template='plotly_white',
        height=600,
        showlegend=False
    )

    fig.show()
else:
    print("Category performance dashboard skipped - no category data available")

In [ ]:
# Create category performance dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Revenue Share', 'Average Order Value', 'Items per Order', 'Revenue per Product'),
    specs=[[{"type": "pie"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

# Revenue share pie chart
top5_categories = category_metrics.head(5)
others_revenue = category_metrics.iloc[5:]['total_revenue'].sum()
pie_data = pd.concat([
    top5_categories[['product_category_name', 'total_revenue']],
    pd.DataFrame([{'product_category_name': 'Others', 'total_revenue': others_revenue}])
])

fig.add_trace(go.Pie(
    labels=pie_data['product_category_name'].str.title(),
    values=pie_data['total_revenue'],
    textinfo='label+percent'
), row=1, col=1)

# Average Order Value by category
fig.add_trace(go.Bar(
    x=top5_categories['avg_order_value'],
    y=top5_categories['product_category_name'].str.title(),
    orientation='h',
    marker_color=BUSINESS_COLORS['success'],
    name='AOV'
), row=1, col=2)

# Items per order
fig.add_trace(go.Bar(
    x=top5_categories['items_per_order'],
    y=top5_categories['product_category_name'].str.title(),
    orientation='h',
    marker_color=BUSINESS_COLORS['info'],
    name='Items/Order'
), row=2, col=1)

# Revenue per product
fig.add_trace(go.Bar(
    x=top5_categories['revenue_per_product'],
    y=top5_categories['product_category_name'].str.title(),
    orientation='h',
    marker_color=BUSINESS_COLORS['warning'],
    name='Revenue/Product'
), row=2, col=2)

fig.update_layout(
    title_text=f"Product Category Performance Dashboard - {ANALYSIS_CONFIG['current_year']}",
    template='plotly_white',
    height=600,
    showlegend=False
)

fig.show()

## Geographic Market Analysis

### Revenue by State

In [ ]:
# Calculate geographic metrics
state_metrics = calculate_geographic_metrics(
    current_period_df, orders_df, customers_df, geographic_level='state'
)

if not state_metrics.empty:
    print("TOP STATES BY REVENUE:")
    print("=" * 50)
    for idx, row in state_metrics.head(10).iterrows():
        print(f"{row['customer_state']:3} ${row['total_revenue']:>10,.2f} ({row['revenue_share_pct']:4.1f}%) | {row['unique_customers']:>4,} customers")

    # Create geographic revenue map
    fig = px.choropleth(
        state_metrics,
        locations='customer_state',
        color='total_revenue',
        locationmode='USA-states',
        scope='usa',
        title=f'Revenue by State - {ANALYSIS_CONFIG["current_year"]}',
        color_continuous_scale='Blues',
        labels={'total_revenue': 'Revenue (USD)'},
        hover_data=['total_orders', 'unique_customers', 'avg_order_value']
    )

    fig.update_layout(
        template='plotly_white',
        height=500
    )

    fig.show()
    
    # Top states bar chart
    top_states = state_metrics.head(15)
    
    fig = go.Figure(data=[
        go.Bar(
            x=top_states['total_revenue'],
            y=top_states['customer_state'],
            orientation='h',
            marker_color=BUSINESS_COLORS['primary'],
            text=[f'${x:,.0f}' for x in top_states['total_revenue']],
            textposition='outside'
        )
    ])
    
    fig.update_layout(
        title=f'Top 15 States by Revenue - {ANALYSIS_CONFIG["current_year"]}',
        xaxis_title='Revenue (USD)',
        yaxis_title='State',
        yaxis={'categoryorder': 'total ascending'},
        template='plotly_white',
        height=500
    )
    
    fig.show()
else:
    print("Geographic data not available for analysis")

## Customer Experience Analysis

### Customer Satisfaction Metrics

In [ ]:
# Calculate customer experience metrics
cx_metrics = calculate_customer_experience_metrics(current_period_df, reviews_df)

print("CUSTOMER EXPERIENCE SUMMARY")
print("=" * 50)
if 'avg_review_score' in cx_metrics:
    print(f"Average Review Score: {cx_metrics['avg_review_score']:.2f}/5.0")
    print(f"Satisfaction Rate (4-5 stars): {cx_metrics['satisfaction_rate_pct']:.1f}%")

if 'avg_delivery_days' in cx_metrics:
    print(f"Average Delivery Time: {cx_metrics['avg_delivery_days']:.1f} days")
    if 'fast_delivery_rate_pct' in cx_metrics:
        print(f"Fast Delivery Rate (≤3 days): {cx_metrics['fast_delivery_rate_pct']:.1f}%")

# Create customer experience dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Review Score Distribution', 'Delivery Speed Distribution', 
                   'Delivery vs Satisfaction', 'Monthly Satisfaction Trend'),
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "bar"}, {"type": "scatter"}]]
)

# Review score distribution
if 'review_score_distribution' in cx_metrics:
    scores = list(cx_metrics['review_score_distribution'].keys())
    percentages = [v * 100 for v in cx_metrics['review_score_distribution'].values()]
    
    fig.add_trace(go.Bar(
        x=scores,
        y=percentages,
        marker_color=BUSINESS_COLORS['success'],
        name='Review Distribution'
    ), row=1, col=1)

# Delivery speed distribution
if 'delivery_speed_distribution' in cx_metrics:
    speed_labels = list(cx_metrics['delivery_speed_distribution'].keys())
    speed_values = [v * 100 for v in cx_metrics['delivery_speed_distribution'].values()]
    
    fig.add_trace(go.Pie(
        labels=speed_labels,
        values=speed_values,
        name='Delivery Speed'
    ), row=1, col=2)

# Delivery speed vs satisfaction
if 'delivery_speed_vs_satisfaction' in cx_metrics:
    speed_sat_data = cx_metrics['delivery_speed_vs_satisfaction']['mean']
    categories = list(speed_sat_data.keys())
    satisfaction = list(speed_sat_data.values())
    
    fig.add_trace(go.Bar(
        x=categories,
        y=satisfaction,
        marker_color=BUSINESS_COLORS['info'],
        name='Avg Review Score'
    ), row=2, col=1)

fig.update_layout(
    title_text=f"Customer Experience Dashboard - {ANALYSIS_CONFIG['current_year']}",
    template='plotly_white',
    height=600,
    showlegend=False
)

fig.show()

### Review Score Analysis

In [ ]:
# Detailed review score analysis
review_data = current_period_df.dropna(subset=['review_score'])

if not review_data.empty:
    # Calculate review score by category
    category_reviews = review_data.groupby('product_category_name')['review_score'].agg(['mean', 'count']).reset_index()
    category_reviews = category_reviews[category_reviews['count'] >= 10]  # Filter for statistical significance
    category_reviews = category_reviews.sort_values('mean', ascending=False)
    
    print("REVIEW SCORES BY PRODUCT CATEGORY:")
    print("=" * 50)
    for _, row in category_reviews.head(10).iterrows():
        print(f"{row['product_category_name'].title():25} {row['mean']:.2f}/5.0 ({int(row['count']):>3} reviews)")
    
    # Create review score by category chart
    fig = go.Figure(data=[
        go.Bar(
            y=category_reviews['product_category_name'].str.title(),
            x=category_reviews['mean'],
            orientation='h',
            marker_color=BUSINESS_COLORS['success'],
            text=[f'{x:.2f}' for x in category_reviews['mean']],
            textposition='outside'
        )
    ])
    
    fig.update_layout(
        title=f'Average Review Score by Product Category - {ANALYSIS_CONFIG["current_year"]}',
        xaxis_title='Average Review Score (1-5)',
        yaxis_title='Product Category',
        yaxis={'categoryorder': 'total ascending'},
        template='plotly_white',
        height=400,
        xaxis_range=[0, 5]
    )
    
    fig.show()
else:
    print("Review data not available for analysis")

## Operational Performance

### Order Fulfillment Metrics

In [ ]:
# Generate comprehensive summary
summary = generate_metrics_summary(
    current_period_df, products_df, orders_df, customers_df, reviews_df,
    f"{ANALYSIS_CONFIG['current_year']} Analysis"
)

# Create executive summary
print("EXECUTIVE SUMMARY")
print("=" * 60)
print(f"Analysis Period: {ANALYSIS_CONFIG['current_year']}")
print(f"Data Coverage: {current_period_df['order_purchase_timestamp'].min().strftime('%B %Y')} - {current_period_df['order_purchase_timestamp'].max().strftime('%B %Y')}")
print()

# Financial Performance
print("FINANCIAL PERFORMANCE:")
rev_metrics = summary.get('revenue', {})
print(f"  • Total Revenue: ${rev_metrics.get('total_revenue', 0):,.2f}")
print(f"  • Revenue Growth: {growth_metrics.get('revenue_growth_pct', 0):+.1f}% YoY")
print(f"  • Average Order Value: ${rev_metrics.get('avg_order_value', 0):,.2f}")
print(f"  • Total Orders: {rev_metrics.get('total_orders', 0):,}")
print()

# Product Performance
print("PRODUCT PERFORMANCE:")
if not category_metrics.empty and len(category_metrics) > 0:
    top_category = category_metrics.iloc[0]
    print(f"  • Top Category: {top_category['product_category_name'].title()}")
    print(f"  • Top Category Revenue: ${top_category['total_revenue']:,.2f} ({top_category['revenue_share_pct']:.1f}%)")
    print(f"  • Categories Analyzed: {len(category_metrics)}")
else:
    print("  • Product category data not available for analysis")
    print(f"  • Total Products Sold: {current_period_df['product_id'].nunique():,}")
print()

# Customer Experience
print("CUSTOMER EXPERIENCE:")
cx = summary.get('customer_experience', {})
if 'avg_review_score' in cx:
    print(f"  • Average Review Score: {cx['avg_review_score']:.2f}/5.0")
    print(f"  • Customer Satisfaction: {cx.get('satisfaction_rate_pct', 0):.1f}%")
if 'avg_delivery_days' in cx:
    print(f"  • Average Delivery Time: {cx['avg_delivery_days']:.1f} days")
    print(f"  • Fast Delivery Rate: {cx.get('fast_delivery_rate_pct', 0):.1f}%")
print()

# Operational Excellence
print("OPERATIONAL EXCELLENCE:")
ops = summary.get('operations', {})
if 'delivery_rate_pct' in ops:
    print(f"  • Delivery Success Rate: {ops['delivery_rate_pct']:.1f}%")
    print(f"  • Order Fulfillment Rate: {ops.get('fulfillment_rate_pct', 0):.1f}%")
    print(f"  • Cancellation Rate: {ops.get('cancellation_rate_pct', 0):.1f}%")
print()

# Geographic Reach
print("MARKET REACH:")
if 'state_metrics' in locals() and not state_metrics.empty:
    print(f"  • States Served: {len(state_metrics)}")
    top_state = state_metrics.iloc[0]
    print(f"  • Top State: {top_state['customer_state']} (${top_state['total_revenue']:,.2f})")
    print(f"  • Geographic Concentration: Top 5 states = {state_metrics.head(5)['revenue_share_pct'].sum():.1f}% of revenue")
else:
    print("  • Geographic analysis not available")
print()

print("KEY INSIGHTS & RECOMMENDATIONS:")
print("=" * 60)

# Growth insights
if growth_metrics.get('revenue_growth_pct', 0) > 0:
    print("✓ GROWTH: Positive revenue growth indicates healthy business expansion")
else:
    print("⚠ ATTENTION: Negative revenue growth requires strategic review")

# Customer satisfaction insights
avg_score = cx.get('avg_review_score', 0)
if avg_score >= 4.0:
    print("✓ SATISFACTION: High customer satisfaction scores support retention")
elif avg_score >= 3.5:
    print("→ OPPORTUNITY: Moderate satisfaction - focus on service improvements")
else:
    print("⚠ PRIORITY: Low satisfaction requires immediate attention")

# Delivery performance insights
avg_delivery = cx.get('avg_delivery_days', 0)
if avg_delivery <= 5:
    print("✓ LOGISTICS: Fast delivery times enhance customer experience")
elif avg_delivery <= 10:
    print("→ EFFICIENCY: Consider optimizing delivery operations")
else:
    print("⚠ LOGISTICS: Extended delivery times may impact satisfaction")

# Product diversity insights
if not category_metrics.empty and len(category_metrics) > 0:
    top_category_share = category_metrics.iloc[0]['revenue_share_pct']
    if top_category_share > 50:
        print("→ DIVERSIFICATION: High category concentration - consider portfolio expansion")
    else:
        print("✓ PORTFOLIO: Well-diversified product categories reduce risk")

print("\n" + "=" * 60)
print("Analysis completed successfully")
print(f"Report generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")

## Summary & Key Insights

### Executive Summary Dashboard

In [ ]:
# Generate comprehensive summary
summary = generate_metrics_summary(
    current_period_df, products_df, orders_df, customers_df, reviews_df,
    f"{ANALYSIS_CONFIG['current_year']} Analysis"
)

# Create executive summary
print("EXECUTIVE SUMMARY")
print("=" * 60)
print(f"Analysis Period: {ANALYSIS_CONFIG['current_year']}")
print(f"Data Coverage: {current_period_df['order_purchase_timestamp'].min().strftime('%B %Y')} - {current_period_df['order_purchase_timestamp'].max().strftime('%B %Y')}")
print()

# Financial Performance
print("FINANCIAL PERFORMANCE:")
rev_metrics = summary.get('revenue', {})
print(f"  • Total Revenue: ${rev_metrics.get('total_revenue', 0):,.2f}")
print(f"  • Revenue Growth: {growth_metrics.get('revenue_growth_pct', 0):+.1f}% YoY")
print(f"  • Average Order Value: ${rev_metrics.get('avg_order_value', 0):,.2f}")
print(f"  • Total Orders: {rev_metrics.get('total_orders', 0):,}")
print()

# Product Performance
print("PRODUCT PERFORMANCE:")
if not category_metrics.empty:
    top_category = category_metrics.iloc[0]
    print(f"  • Top Category: {top_category['product_category_name'].title()}")
    print(f"  • Top Category Revenue: ${top_category['total_revenue']:,.2f} ({top_category['revenue_share_pct']:.1f}%)")
    print(f"  • Categories Analyzed: {len(category_metrics)}")
print()

# Customer Experience
print("CUSTOMER EXPERIENCE:")
cx = summary.get('customer_experience', {})
if 'avg_review_score' in cx:
    print(f"  • Average Review Score: {cx['avg_review_score']:.2f}/5.0")
    print(f"  • Customer Satisfaction: {cx.get('satisfaction_rate_pct', 0):.1f}%")
if 'avg_delivery_days' in cx:
    print(f"  • Average Delivery Time: {cx['avg_delivery_days']:.1f} days")
    print(f"  • Fast Delivery Rate: {cx.get('fast_delivery_rate_pct', 0):.1f}%")
print()

# Operational Excellence
print("OPERATIONAL EXCELLENCE:")
ops = summary.get('operations', {})
if 'delivery_rate_pct' in ops:
    print(f"  • Delivery Success Rate: {ops['delivery_rate_pct']:.1f}%")
    print(f"  • Order Fulfillment Rate: {ops.get('fulfillment_rate_pct', 0):.1f}%")
    print(f"  • Cancellation Rate: {ops.get('cancellation_rate_pct', 0):.1f}%")
print()

# Geographic Reach
print("MARKET REACH:")
if not state_metrics.empty:
    print(f"  • States Served: {len(state_metrics)}")
    top_state = state_metrics.iloc[0]
    print(f"  • Top State: {top_state['customer_state']} (${top_state['total_revenue']:,.2f})")
    print(f"  • Geographic Concentration: Top 5 states = {state_metrics.head(5)['revenue_share_pct'].sum():.1f}% of revenue")
print()

print("KEY INSIGHTS & RECOMMENDATIONS:")
print("=" * 60)

# Growth insights
if growth_metrics.get('revenue_growth_pct', 0) > 0:
    print("✓ GROWTH: Positive revenue growth indicates healthy business expansion")
else:
    print("⚠ ATTENTION: Negative revenue growth requires strategic review")

# Customer satisfaction insights
avg_score = cx.get('avg_review_score', 0)
if avg_score >= 4.0:
    print("✓ SATISFACTION: High customer satisfaction scores support retention")
elif avg_score >= 3.5:
    print("→ OPPORTUNITY: Moderate satisfaction - focus on service improvements")
else:
    print("⚠ PRIORITY: Low satisfaction requires immediate attention")

# Delivery performance insights
avg_delivery = cx.get('avg_delivery_days', 0)
if avg_delivery <= 5:
    print("✓ LOGISTICS: Fast delivery times enhance customer experience")
elif avg_delivery <= 10:
    print("→ EFFICIENCY: Consider optimizing delivery operations")
else:
    print("⚠ LOGISTICS: Extended delivery times may impact satisfaction")

# Product diversity insights
if not category_metrics.empty:
    top_category_share = category_metrics.iloc[0]['revenue_share_pct']
    if top_category_share > 50:
        print("→ DIVERSIFICATION: High category concentration - consider portfolio expansion")
    else:
        print("✓ PORTFOLIO: Well-diversified product categories reduce risk")

print("\n" + "=" * 60)
print("Analysis completed successfully")
print(f"Report generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")